In [1]:
import os
os.chdir("/kaggle/input/ptbxl-all-diag") # Перейдем в Input (только для чтения!)
!ls # Посмотреть содержимое

X_train_ptbxl_all.npy	X_val_ptbxl_diag.npy	y_val_ptbxl_all.npy
X_train_ptbxl_diag.npy	y_train_ptbxl_all.npy	y_val_ptbxl_diag.npy
X_val_ptbxl_all.npy	y_train_ptbxl_diag.npy


In [2]:
os.chdir("/kaggle/working/") # Перейдем в Output
!ls

__notebook_source__.ipynb


In [3]:
# Для работы с данными
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt   # plotting
import seaborn as sns   # plotting heatmap
import tensorflow as tf
import math
import pickle

# Для работы с моделями
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
from keras.models import load_model, save_model

# Для метрик
from keras import backend as K
from keras.metrics import AUC, Recall, Precision, Accuracy, TruePositives, TrueNegatives, FalsePositives, FalseNegatives
from sklearn.metrics import fbeta_score, precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn.metrics import auc, roc_curve, classification_report
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, f1_score

# Код из учебника Жерона
K = keras.backend

# # 71 класс SCP_ECG в алфавитном порядке для категории 'all'
labels = ['1AVB', '2AVB', '3AVB', 'ABQRS', 'AFIB', 'AFLT', 
          'ALMI', 'AMI', 'ANEUR', 'ASMI', 'BIGU', 'CLBBB', 
          'CRBBB', 'DIG', 'EL', 'HVOLT', 'ILBBB', 'ILMI', 
          'IMI', 'INJAL', 'INJAS', 'INJIL', 'INJIN','INJLA', 
          'INVT', 'IPLMI', 'IPMI', 'IRBBB', 'ISC_', 'ISCAL', 
          'ISCAN', 'ISCAS', 'ISCIL', 'ISCIN', 'ISCLA', 'IVCD', 
          'LAFB', 'LAO/LAE', 'LMI', 'LNGQT', 'LOWT', 'LPFB', 
          'LPR', 'LVH', 'LVOLT', 'NDT', 'NORM', 'NST_', 
          'NT_', 'PAC', 'PACE', 'PMI', 'PRC(S)', 'PSVT', 
          'PVC', 'QWAVE', 'RAO/RAE', 'RVH', 'SARRH', 'SBRAD', 
          'SEHYP', 'SR', 'STACH', 'STD_', 'STE_', 'SVARR', 
          'SVTAC', 'TAB_', 'TRIGU', 'VCLVH', 'WPW']

# 44 класса SCP_ECG в алфавитном порядке для категории 'diag'
# labels = ['1AVB', '2AVB', '3AVB',
#           'ALMI', 'AMI', 'ANEUR', 'ASMI', 'CLBBB', 
#           'CRBBB', 'DIG', 'EL', 'ILBBB', 'ILMI', 
#           'IMI', 'INJAL', 'INJAS', 'INJIL', 'INJIN','INJLA', 
#           'IPLMI', 'IPMI', 'IRBBB', 'ISC_', 'ISCAL', 
#           'ISCAN', 'ISCAS', 'ISCIL', 'ISCIN', 'ISCLA', 'IVCD', 
#           'LAFB', 'LAO/LAE', 'LMI', 'LNGQT', 'LPFB', 
#           'LVH', 'NDT', 'NORM', 'NST_', 
#           'PMI', 
#           'RAO/RAE', 'RVH', 
#           'SEHYP', 'WPW']


class ExponentialLearningRate(keras.callbacks.Callback):
    def __init__(self, factor):
        self.factor = factor
        self.rates = []
        self.losses = []
    def on_batch_end(self, batch, logs):
        self.rates.append(K.get_value(self.model.optimizer.learning_rate))
        self.losses.append(logs["loss"])
        K.set_value(self.model.optimizer.learning_rate, self.model.optimizer.learning_rate * self.factor)

def find_learning_rate(model, X, y, epochs=1, batch_size=32, min_rate=10**-5, max_rate=10):
    init_weights = model.get_weights()
    iterations = math.ceil(len(X) / batch_size) * epochs
    factor = np.exp(np.log(max_rate / min_rate) / iterations)
    init_lr = K.get_value(model.optimizer.learning_rate)
    K.set_value(model.optimizer.learning_rate, min_rate)
    exp_lr = ExponentialLearningRate(factor)
    history = model.fit(X, y, epochs=epochs, batch_size=batch_size,
                        callbacks=[exp_lr])
    K.set_value(model.optimizer.learning_rate, init_lr)
    model.set_weights(init_weights)
    return exp_lr.rates, exp_lr.losses

def plot_lr_vs_loss(rates, losses):
    plt.plot(rates, losses)
    plt.gca().set_xscale('log')
    plt.hlines(min(losses), min(rates), max(rates))
    plt.axis([min(rates), max(rates), min(losses), (losses[0] + min(losses)) / 2])
    plt.xlabel("Learning rate")
    plt.ylabel("Loss")

class ExponentialLearningRate(keras.callbacks.Callback):
    def __init__(self, factor):
        self.factor = factor
        self.rates = []
        self.losses = []
    def on_epoch_begin(self, epoch, logs=None):
        self.prev_loss = 0
    def on_batch_end(self, batch, logs=None):
        batch_loss = logs["loss"] * (batch + 1) - self.prev_loss * batch
        self.prev_loss = logs["loss"]
        self.rates.append(K.get_value(self.model.optimizer.learning_rate))
        self.losses.append(batch_loss)
        K.set_value(self.model.optimizer.learning_rate, self.model.optimizer.learning_rate * self.factor)

class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None,
                 last_iterations=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate / 1000
        self.iteration = 0
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return ((rate2 - rate1) * (self.iteration - iter1)
                / (iter2 - iter1) + rate1)
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0, self.half_iteration, self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                     self.max_rate, self.start_rate)
        else:
            rate = self._interpolate(2 * self.half_iteration, self.iterations,
                                     self.start_rate, self.last_rate)
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, rate)

class attention(layers.Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1),
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1),
                               initializer='zeros', trainable=True)
        super(attention, self).build(input_shape)

    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

def ATI_CNN(num_classes, multi_label_classifier=False):
    activation = keras.activations.sigmoid if multi_label_classifier else keras.activations.softmax

    def Conv_block_1(filters):
        return keras.Sequential([
            layers.Conv1D(filters=filters, kernel_size=3, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Conv1D(filters=filters, kernel_size=3, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.MaxPooling1D(pool_size=3, strides=3),
            layers.Dropout(rate=0.2)
        ])

    def Conv_block_2(filters):
        return keras.Sequential([
            layers.Conv1D(filters=filters, kernel_size=3, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Conv1D(filters=filters, kernel_size=3, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Conv1D(filters=filters, kernel_size=3, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.MaxPooling1D(pool_size=3, strides=3),
            layers.Dropout(rate=0.2)
        ])

    shape = (1000, 12)
    input = keras.Input(shape=shape, name="ECG")

    x = Conv_block_1(64)(input)
    x = Conv_block_1(128)(x)

    x = Conv_block_2(256)(x)
    x = Conv_block_2(256)(x)
    x = Conv_block_2(256)(x)

    x = layers.LSTM(units=32, return_sequences=True)(x)
    x = layers.LSTM(units=32, return_sequences=True)(x)

    x = attention()(x)

    output = layers.Dense(units=num_classes, activation=activation, name='output')(x)

    model = keras.Model(inputs=[input], outputs=[output])

    return model

def M_ATI_CNN(num_classes, multi_label_classifier=False):
    activation = layers.Activation(keras.activations.sigmoid) if multi_label_classifier else layers.Activation(keras.activations.softmax)

    def Conv_block_1(filters):
        return keras.Sequential([
            layers.Conv1D(filters=filters, kernel_size=3, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Conv1D(filters=filters, kernel_size=3, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.MaxPooling1D(pool_size=3, strides=3),
            layers.Dropout(rate=0.2)
        ])

    def Conv_block_2(filters):
        return keras.Sequential([
            layers.Conv1D(filters=filters, kernel_size=3, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Conv1D(filters=filters, kernel_size=3, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Conv1D(filters=filters, kernel_size=3, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.MaxPooling1D(pool_size=3, strides=3),
            layers.Dropout(rate=0.2)
        ])

    shape = (1000, 12)
    input = keras.Input(shape=shape, name="ECG")

    output_modules = list()

    for i in range(num_classes):
      output_modules.append(keras.Sequential([
          Conv_block_1(64),
          Conv_block_1(128),
          Conv_block_2(256),
          Conv_block_2(256),
          Conv_block_2(256)
      ])(input))

      output_modules[-1] = layers.Bidirectional(layers.LSTM(units=32, return_sequences=True))(output_modules[-1])

      output_modules[-1] = attention()(output_modules[-1])
      output_modules[-1] = layers.Dense(units=1)(output_modules[-1])

    cat_outputs =  keras.layers.Concatenate()(output_modules)

    output = activation(cat_outputs)

    model = keras.Model(inputs=[input], outputs=[output])

    return model

def DNN(num_classes, multi_label_classifier=False):
    activation = keras.activations.sigmoid if multi_label_classifier else keras.activations.softmax

    def Conv_block_1(filters, kernel_size, pool_size):
        return keras.Sequential([
            layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Dropout(rate=0.1),
            layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.MaxPooling1D(pool_size=pool_size)
        ]), layers.MaxPooling1D(pool_size=pool_size)

    def Conv_block_2(filters, kernel_size, pool_size):
        return keras.Sequential([
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Dropout(rate=0.1),
            layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Dropout(rate=0.1),
            layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.MaxPooling1D(pool_size=pool_size)
        ]), layers.MaxPooling1D(pool_size=pool_size)

    shape = (1000, 12)
    input = keras.Input(shape=shape, name="ECG")

    x = keras.Sequential([
        layers.Conv1D(filters=32, kernel_size=16, padding='same'),
        layers.BatchNormalization(),
        layers.ReLU()
    ])(input)

    x1, x2 = Conv_block_1(kernel_size=16, filters=32, pool_size=2)
    x = layers.Concatenate()([x1(x), x2(x)])

    for filters, kernel_size, pool_size in zip(
        [32, 32, 32, 64, 64, 64, 64, 96, 96, 96, 96],
        [16, 16, 16, 16, 16,  8,  8,  8,  8,  8,  8],
        [ 1,  2,  1,  2,  1,  2,  1,  2,  1,  2,  1]
    ):
        x1, x2 = Conv_block_2(filters=filters, kernel_size=kernel_size, pool_size=pool_size)
        x = layers.Concatenate()([x1(x), x2(x)])

    x = keras.Sequential([
        layers.BatchNormalization(),
        layers.ReLU()
    ])(x)

    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True), merge_mode='sum')(x)
    x = layers.GlobalMaxPooling1D()(x)

    x = layers.Dense(units=64, activation=keras.activations.relu)(x)
    output = layers.Dense(units=num_classes, activation=activation, name='output')(x)

    model = keras.Model(inputs=[input], outputs=[output])

    return model

# Загрузка ptbxl
def load_ptbxl(task):
  os.chdir("/kaggle/input/ptbxl-all-diag")
  if task == 'diag':
    X_train = np.load('X_train_ptbxl_diag.npy')
    y_train = np.load('y_train_ptbxl_diag.npy')
    X_test = np.load('X_val_ptbxl_diag.npy')
    y_test = np.load('y_val_ptbxl_diag.npy')
  elif task == 'all':
    X_train = np.load('X_train_ptbxl_all.npy')
    y_train = np.load('y_train_ptbxl_all.npy')
    X_test = np.load('X_val_ptbxl_all.npy')
    y_test = np.load('y_val_ptbxl_all.npy')
  os.chdir("/kaggle/working/")
  return X_train, y_train, X_test, y_test

# Сериализация
def sp(obj, name): # sp = serialization_pickle ;name = *.pkl
    with open(name, 'wb') as pickle_out:
        pickle.dump(obj, pickle_out)

# Компиляция и обучение модели
def AUC_Keras(y_true, y_pred):
    auc = keras.metrics.AUC(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

# Компиляция и обучение модели
def compile_fit(model, X_train, y_train, X_val = None, y_val = None, validation_split = 0.0, batch_size = None, epochs = None, early_stopping = None, model_checkpoint = None, onecycle = None):
  model.compile(loss = keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.experimental.AdamW(),
                metrics=['AUC'])

  if X_val == None:
    history = model.fit(X_train, y_train,
                        epochs = epochs,
                        batch_size = batch_size,
                        validation_data = None,
                        validation_split=validation_split,
                        callbacks=[onecycle, model_checkpoint, early_stopping])
    sp(history, 'history.pkl')
  else:
    history = model.fit(X_train, y_train,
                        epochs = epochs,
                        batch_size = batch_size,
                        validation_data = (X_val, y_val),
                        validation_split=0.0,
                        callbacks=[onecycle, model_checkpoint, early_stopping])
    sp(history, 'history.pkl')
  return history

# График loss и accuracy
def plot_loss_and_accuracy_curves(_history):
  fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(18,6))
  axs[0].plot(_history.history['loss'], color='b', label='Training loss')
  axs[0].plot(_history.history['val_loss'], color='r', label='Validation loss')
  axs[0].set_title("Loss")
  axs[0].legend(loc='best', shadow=True)
  axs[1].plot(_history.history['auc'], color='b', label='Training accuracy')
  axs[1].plot(_history.history['val_auc'], color='r', label='Validation accuracy')
  axs[1].set_title("AUC")
  axs[1].legend(loc='best', shadow=True)
  plt.show()

# Работа с моделями lstm и lstm_bidir
def type_comp_fit_save_model_score(X_train, y_train, X_test, y_test, type_model, save_name, index_model_task, batch_size, epochs, onecycle):
  # количество классов
  num_classes = y_train.shape[1]

  # Выбор архитектуры модели
  if type_model == 'lstm':
    inputs = keras.Input(shape=(1000, 12))
    x = layers.LSTM(units=256,
                    return_sequences=True,
                    stateful=False,
                    unroll=False)(inputs)
    x = layers.LeakyReLU()(x)
    x = layers.LSTM(units=256,
                    return_sequences=True, # True if concat
                    stateful=False,
                    unroll=False)(x)
    x = layers.LeakyReLU()(x)
    avg_pool = layers.GlobalAveragePooling1D()(x)
    max_pool = layers.GlobalMaxPooling1D()(x)
    concat = layers.Concatenate()([avg_pool, max_pool])
    outputs = layers.Dense(units=num_classes, activation='sigmoid')(concat)
    model = keras.Model(inputs=inputs, outputs=outputs)
    print(model.summary())

    # Реализация раннего прекращения.
    checkpoint_filepath = './checkpoint_lstm/'
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                          save_weights_only=True,
                                                          save_best_only=True)
    early_stopping = keras.callbacks.EarlyStopping(patience=epochs//10,
                                                  restore_best_weights=True)
  elif type_model == 'lstm_bidir':
    inputs = keras.Input(shape=(1000, 12))
    x = layers.Bidirectional(layers.LSTM(units=256,
                             return_sequences = True,
                             stateful = False,
                             unroll = False))(inputs)
    x = layers.LeakyReLU()(x)
    x = layers.Bidirectional(layers.LSTM(units=256,
                             return_sequences = True,
                             stateful = False,
                             unroll = False))(x)
    x = layers.LeakyReLU()(x)
    avg_pool = layers.GlobalAveragePooling1D()(x)
    max_pool = layers.GlobalMaxPooling1D()(x)
    concat = layers.Concatenate()([avg_pool, max_pool])
    outputs = layers.Dense(units=num_classes, activation='sigmoid')(concat)
    model = keras.Model(inputs=inputs, outputs=outputs)

    model.build(input_shape = (None, 1000, 12)) # `input_shape` is the shape of the input data

    print(model.summary())

    # Реализация раннего прекращения.
    checkpoint_filepath = './checkpoint_lstm_bidir/'
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                          save_weights_only=True,
                                                          save_best_only=True)
    early_stopping = keras.callbacks.EarlyStopping(patience=epochs//10,
                                                  restore_best_weights=True)

  # Обучение
  History = compile_fit(model, X_train, y_train, validation_split=0.1, batch_size=batch_size, epochs=epochs, early_stopping=early_stopping, model_checkpoint=model_checkpoint, onecycle=onecycle)

  # Сохранение модели
  model.save(save_name)

  # Построение графика
  plot_loss_and_accuracy_curves(History)

  return model

# Подсчет метрик f1, f2, g1 и g2 (где 1 и 2 значения параметра beta) для каждого класса в задаче многозначной классификации (multilabel classication)
def calc_metrics(y_true, y_pred, thresholds, labels): 
  num_classes = y_true.shape[1] # n x m, где m - количество классов (применялось one-hot кодирование)
  table_scores = pd.DataFrame({'label': labels[:num_classes], 
                               'f1': pd.Series([]), 
                               'f2': pd.Series([]), 
                               'g1': pd.Series([]), 
                               'g2': pd.Series([])}, index = range(1, num_classes + 1))
    
  for i, conf_matr in enumerate(multilabel_confusion_matrix(y_true, y_pred > thresholds)):
    
    label_index = i + 1 # i + 1, потому что в table_scores индексация начинается с 1                            
    true = y_true[:, i]
    pred = y_pred[:, i]
    table_scores.loc[label_index, 'f1'] = fbeta_score(true, pred > thresholds[i], beta = 1) 
    table_scores.loc[label_index, 'f2'] = fbeta_score(true, pred > thresholds[i], beta = 2)
                                
    TP = conf_matr[1, 1]
    FP = conf_matr[0, 1]
    TN = conf_matr[0, 0]
    FN = conf_matr[1, 0]                        
    beta = 1
    g1 = TP/(TP + FP + beta * FN) # beta = 1
    beta = 2
    g2 = TP/(TP + FP + beta * FN) # beta = 2
    table_scores.loc[label_index, 'g1'] = g1
    table_scores.loc[label_index, 'g2'] = g2
                                
  return table_scores

def plot_roc_and_cm_for_multi_label_clf(y_test, y_pred, labels):
  fig, axes = plt.subplots(12, 6, figsize=(100, 100))
  model_thresholds = list()

  for i, label in enumerate(labels):
    fpr, tpr, thresholds = roc_curve(y_test[:, i], y_pred[:, i])
    roc_auc = auc(fpr, tpr)

    idx = (tpr - fpr + 1).argmax()
    model_thresholds.append(thresholds[idx])

    ax = axes[i // 6, i % 6]

    ax.plot(fpr, tpr, color='darkorange',
            label=f'{label} vs the rest (area = {roc_auc:0.2f})', zorder=0)
    ax.plot([0, 1], [0, 1], color='k', linestyle='--', label="chance level (AUC = 0.5)")
    ax.scatter([fpr[idx]], [tpr[idx]], color='g', marker='x', label='optimal value', zorder=1)
    ax.set(xlim=[0.0, 1.0], ylim=[0.0, 1.05],
          xlabel='False Positive Rate',
          ylabel='True Positive Rate',
          title=f'{label}',)
    ax.legend(loc="lower right")

  fig.delaxes(axes[-1, -1])
  fig.suptitle('lstm_bidir ROC кривые', fontsize=20)
  fig.savefig('/lstm_bidir/lstm_bidir_ROC_curves.png')

  model_thresholds = np.array(model_thresholds)

  fig, axes = plt.subplots(12, 6, figsize=(100, 100))

  for i, (conf_matr, label) in enumerate(zip(multilabel_confusion_matrix(y_test, y_pred > model_thresholds), labels)):
    ax = axes[i // 6, i % 6]
    sns.heatmap(conf_matr,
              square=True,
              annot=True,
              fmt='.0f',
              cmap=plt.cm.Blues,
              ax=ax)
    ax.set_xticklabels([f'not {label}', label])
    ax.set_yticklabels([f'not {label}', label], rotation=45, ha='right')
    ax.set(xlabel="Predicted label", ylabel="True label")

  fig.delaxes(axes[-1, -1])
  fig.suptitle('lstm bidir confusion matrix', fontsize=20)

  fig.savefig('/lstm_bidir/lstm_bidir_cm.png')

  print(classification_report(y_test, y_pred > model_thresholds, target_names=labels))
  pass
  return model_thresholds

def experiment(X_train, y_train, X_test, y_test, batch_size, epochs, type_model, save, index_model_task, labels):
  onecycle = OneCycleScheduler(math.ceil(len(X_train) / batch_size) * epochs, max_rate=0.0001)
  model = type_comp_fit_save_model_score(X_train, y_train, X_test, y_test, type_model = type_model, save_name = save, index_model_task = index_model_task, batch_size=batch_size, epochs=epochs, onecycle=onecycle)
  print(model.evaluate(X_test, y_test))
  y_pred = model.predict(X_test)
  model_thresholds = plot_roc_and_cm_for_multi_label_clf(y_test, y_pred, labels)
  table = calc_metrics(y_test, y_pred, model_thresholds, labels)
  sp(table, 'table.pkl')
  return model, table

tf.random.set_seed(42)
%matplotlib inline

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

### Модель lstm_bidir на категории 'all'

In [ ]:
X_train, y_train, X_test, y_test = load_ptbxl(task = 'all')
save = 'lstm_bidir/model_lstm_bidir_all'
lstm_bidir, table = experiment(X_train, y_train, X_test, y_test, 256, 400, 'lstm_bidir', save, 'lstm_bidir_all', labels)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1000, 12)]   0           []                               
                                                                                                  
 bidirectional (Bidirectional)  (None, 1000, 512)    550912      ['input_1[0][0]']                
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 1000, 512)    0           ['bidirectional[0][0]']          
                                                                                                  
 bidirectional_1 (Bidirectional  (None, 1000, 512)   1574912     ['leaky_re_lu[0][0]']            
 )                                                                                            

In [ ]:
!cd /kaggle/working
#!ls
!zip -r all_bidir.zip /kaggle/working

In [ ]:
# Загрузка данных
X_train, y_train, X_test, y_test = load_ptbxl(task = 'all')

# Обучение модели
model = M_ATI_CNN(num_classes=71, multi_label_classifier=True)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss=[keras.losses.BinaryCrossentropy()],
    metrics=[keras.metrics.AUC()]
)
callbacks = [
    keras.callbacks.EarlyStopping(
        # Прекратить обучение если `val_loss` больше не улучшается
        monitor='val_auc',
        mode='max',
        # "больше не улучшается" определим как "не лучше чем min_delta и меньше"
        min_delta=1e-2,
        # "больше не улучшается" далее определим как "как минимум в течение patience эпох"
        patience=5,
        verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        filepath='M_ATI_CNN_ML.h5',
        # Путь по которому нужно сохранить модель
        # Два параметра ниже значат что мы перезапишем
        # текущий чекпоинт в том и только в том случае, когда
        # улучится значение `val_loss`.
        save_best_only=True,
        monitor='val_auc',
        mode='max',
        verbose=1
    )
]
history = model.fit(X_train, y_train,
        validation_data=0.1,
        epochs=100,
        batch_size=64,
        callbacks=callbacks,
        shuffle=True)



In [ ]:
!cd /kaggle/working
#!ls
!zip -r superdiag.zip /kaggle/working